importing all necessary libraries

In [174]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import matplotlib.dates as mdates
import random
import datetime

In [175]:
class datagenerator:
    def __init__(self,num_transport,num_warehouses,num_customer_sites,num_routes,incompatible,days,capacity,time_transport,freight,quantity,cost,max_day,max_weight,outflow_warehouse,inflow_customersite,p):
        self.model=ConcreteModel()
        self.model.num_transport=num_transport
        self.model.num_warehouses=num_warehouses
        self.model.num_customer_sites=num_customer_sites
        self.model.num_routes=num_routes
        self.model.days=days
        self.model.capacity=capacity
        self.model.time_transport=time_transport
        self.model.incompatible=incompatible
        self.model.freight=freight
        self.model.quantity=quantity
        self.model.cost=cost
        self.model.max_day=max_day
        self.model.max_weight=max_weight
        self.model.p=p
        self.model.outflow_warehouse=outflow_warehouse
        self.model.inflow_customersite=inflow_customersite
    def make_sites_index(self):
        customer_sites=[]
        for i in range(self.model.num_customer_sites):
            customer_sites.append(i)
        self.model.customer_sites=customer_sites
        warehouse_sites=[]
        for i in range(self.model.num_warehouses):
            warehouse_sites.append(i)
        self.model.warehouse_sites=warehouse_sites
    def make_transport_index(self):
        transport=[]
        for i in range(self.model.num_transport):
            transport.append(i)
        self.model.transport=transport
    def compatible(self):
        compatibility=[]
        for i in self.model.routes:
            x={}
            for j in self.model.transport:
                x[j]=1
            for j in self.model.incompatible[i]:
                x[j]=0
            compatibility.append(x)
        self.model.compatibility=compatibility
    def routes_indexing(self):
        self.model.routes=[]
        for i in range(self.model.num_routes):
            self.model.routes.append(i)
    def possible_startdays(self):
        startdays=[]
        for i in range(self.model.max_day+1):
            startdays.append(i)
        self.model.startdays=startdays
    def possible_weights(self):
        weights=[]
        for i in range(self.model.max_weight+1):
            weights.append(i)
        self.model.weights=weights
    def real_quantity(self):
        numbers=[]
        for i in range(max(self.model.quantity)):
            numbers.append(i)
        self.model.numbers = numbers
    def supply_and_demand(self,supply,demand):
        self.model.demand=demand
        self.model.supply=supply
               

In [176]:
def generate_poisson_numbers(temp_time_transport,epsilon,lambda_value):
    length = 0
    samples = []
    while (length < len(temp_time_transport)):
        sample = np.random.poisson(lambda_value)
        if (temp_time_transport[length]*(1-epsilon) <= sample <= temp_time_transport[length]*(1+epsilon)):
            samples.append(sample)
            length+=1
    return samples

test case

In [177]:
num_transport=5
num_warehouses=2
num_customer_sites=2
num_routes=num_warehouses*num_customer_sites
max_day=30

# incompatible=[]
# for i in range(num_routes):
#     j=np.random.randint(0,num_transport-1)
#     temp=random.sample(range(num_transport),j)
#     incompatible.append(temp)
incompatible=[[1],[0],[],[0]]
# days=[]
# for i in range(num_routes):
#     j=np.random.randint(0,5)
#     k=np.random.randint(max_day-5,max_day+1)
#     days.append([j,k])
days=[[3,5],[2,4],[4,6],[1,3]]
routes_idx = ["A-D","A-E","B-E","B-D"]
means_transport = ["Truck (20ft)", "Truck (40ft)", "Rail", "Ship", "Aeroplane"]
time_transport=[1,1,2,5,10]
capacity=[10,15,20,50,100]
# freight=[]
# for i in range(num_routes):
#     j=np.random.randint(1,max(capacity)/2)
#     freight.append(j)
freight=[5,12,8,15]
total_sum=sum(freight)
cost=[50,50,40,30,60]
quantity=[1,1,4,4,4]
max_weight=100
p = 0.5
M = 10000000
outflow_warehouse=[]
for i in range(num_warehouses):
    temp={}
    for j in range(num_customer_sites):
        temp[num_customer_sites*i+j]=1
    outflow_warehouse.append(temp)
inflow_customersite=[]
for i in range(num_customer_sites):
    temp=[]
    for j in range(num_warehouses):
        temp.append(i+num_customer_sites*j)
    inflow_customersite.append(temp) 
# supply=[]
# for i in range(num_warehouses):
#     supply.append(np.random.randint(((total_sum/num_warehouses)+1),2000))
supply=[5,35]
# demand=[]
# for i in range(num_customer_sites):
#     x=0
#     for j in range(num_warehouses):
#         x=x+freight[i+num_customer_sites*j]
#     demand.append(x)
demand=[10,20]

In [178]:
ans_model=datagenerator(num_transport,num_warehouses,num_customer_sites,num_routes,incompatible,days,capacity,time_transport,freight,quantity,cost,max_day,max_weight,outflow_warehouse,inflow_customersite,p)

In [179]:
ans_model.make_sites_index()
ans_model.make_transport_index()
ans_model.routes_indexing()
ans_model.compatible()
ans_model.possible_startdays()
ans_model.possible_weights()
ans_model.real_quantity()
ans_model.supply_and_demand(supply,demand)

In [180]:
ans_model.model.x = pyo.Var(ans_model.model.routes,ans_model.model.transport,ans_model.model.startdays,ans_model.model.numbers,within=Integers,bounds=(0,1))
ans_model.model.y = pyo.Var(ans_model.model.warehouse_sites,ans_model.model.warehouse_sites,ans_model.model.transport,ans_model.model.startdays,ans_model.model.weights,ans_model.model.numbers,within=Integers,bounds=(0,1))
ans_model.model.s = pyo.Var(ans_model.model.warehouse_sites,ans_model.model.startdays,within=Integers,bounds=(0,None))
ans_model.model.d = pyo.Var(ans_model.model.customer_sites,ans_model.model.startdays,within=Integers)
ans_model.model.last = pyo.Var(ans_model.model.routes,within=Integers,bounds=(0,None))
ans_model.model.final = pyo.Var(within=Integers,bounds=(0,None))

model parameters

In [181]:
def sum(t):
    temp = 0
    for i in t:
        temp += i
    return temp

In [182]:
def prod(t):
    temp = 0
    for i in t:
        temp *= i
    return temp

constraints

In [183]:
def rule1(model,i,j,k,n):
    return model.last[i] * model.x[i,j,k,n] >= model.x[i,j,k,n] * (k+model.time_transport[j])

This constraint is to show that the last time of delivery per route will be after all deliveries, and since we will minimize it, this constraint will give the correct value.

In [184]:
def rule2(model,i):
    return model.last[i] >= model.days[i][0]

This shows that the last delivery for every route has to be after the starting of the time window.

In [185]:
def rule3(model,i):
    return model.last[i] <= model.days[i][1]

This shows that the last delivery for every route has to be before the ending of the time window.

In [186]:
def rule4(model,i,j,k,n):
    return model.x[i,j,k,n] * sum(model.x[i,tr,t,r] for tr in model.transport for t in range(k,min(max(model.startdays)+1,k+model.time_transport[tr]+1)) for r in model.numbers) <= 1

While a particular number of a means of transport is transporting the freight, no other means of transport can interfere and start transporting.

In [187]:
def rule5(model,j,k,n):
    return sum(model.x[i,j,k,n] * model.x[i,j,t,n] for i in model.routes for t in range(k,min(max(model.startdays)+1,k+model.time_transport[j]+1))) +  sum(model.y[i,m,j,k,l,n] * model.y[i,m,j,t,l,n] for i in model.warehouse_sites for m in model.warehouse_sites for t in range(k,min(max(model.startdays)+1,k+model.time_transport[j]+1)) for l in model.weights) <= 1

While a particular number of means of transport is transporting somewhere, it cannot start transporting elsewhere unless the transit time has been done.

In [188]:
def rule6(model,i,j,k,n):
    if (model.compatibility[i][j] == 0):
        return model.x[i,j,k,n] == 0
    else:
        return Constraint.Skip

An incompatible means of transport cannot be used.

In [189]:
def rule7(model,i,j,k,n):
    return model.x[i,j,k,n] * model.capacity[j] >= model.x[i,j,k,n] * model.freight[i]

The capacity of transport should exceed the freight to be transported through the route if it is to be used.

In [190]:
def rule8(model,i):
    return model.final >= model.last[i]

This is the last time of delivery overall.

In [191]:
def rule9(model,i,m,j,k,l,n):
    if (l == 0):
        return model.y[i,m,j,k,l,n] == 0
    else:
        return model.y[i,m,j,k,l,n] * l <= model.capacity[j] 

The weight being transported in between warehouses should be less than the capacity.

In [192]:
def rule10(model,st,sd):
    if (sd == 0):
        return model.s[st,sd] == model.supply[st]
    else:
        return model.s[st,sd-1] - sum(model.x[r,t,sd-1,n] * model.freight[r] for t in model.transport for r in model.outflow_warehouse[st].keys() for n in model.numbers) - sum(model.y[st,i,t,sd-1,w,n] * w for i in model.warehouse_sites for t in model.transport for w in model.weights for n in model.numbers) + sum(model.y[i,st,t,sd-1-model.time_transport[t],w,n] * w for i in model.warehouse_sites for t in model.transport for w in model.weights for n in model.numbers if sd >= 1 + model.time_transport[t]) == model.s[st,sd]

This is supply constraint comparing the amount going in and out of the warehouse sites.

In [193]:
def rule11(model,des,sd):
    if (sd == 0):
        return model.d[des,sd] == model.demand[des]
    else:
        return model.d[des,sd-1] - sum(model.x[r,t,sd-1-model.time_transport[t],n] * model.freight[r] for r in model.inflow_customersite[des] for t in model.transport for n in model.numbers if (sd-1-model.time_transport[t]) >= 0) == model.d[des,sd]

This is demand constraint comparing the amount going in and out of the customer_sites.

In [194]:
def rule12(model,i,m,j,k,l,n):
    if (i == m):
        return model.y[i,m,j,k,l,n] == 0
    else:
        return Constraint.Skip

A freight cannot start and end at the same place.

In [195]:
def rule13(model,des,sd):
    if (sd==max(model.startdays)):
        return model.d[des,sd] <= 0
    else:
        return Constraint.Skip

At end of all days, all demand should be sufficed.

In [196]:
def rule14(model,st,sd):
    return model.s[st,sd] >= 0

At all days, the supply at each warehouse site >= 0.

In [197]:
def rule15(model,i):
    return sum(model.x[i,j,k,n] for j in model.transport for k in model.startdays for n in model.numbers) >= 1

Each route has at least one transport operational.

In [198]:
def rule16(model,i,j,k,n):
    if (n >= model.quantity[j]):
        return model.x[i,j,k,n] == 0
    else:
        return Constraint.Skip

The total number of transport being used cannot be more than the number of transport available for every means of transport.

apply constraints

In [199]:
ans_model.model.C1 = pyo.Constraint(ans_model.model.routes, ans_model.model.transport, ans_model.model.startdays, ans_model.model.numbers, rule = rule1)

In [200]:
ans_model.model.C2 = pyo.Constraint(ans_model.model.routes, rule = rule2)

In [201]:
ans_model.model.C3 = pyo.Constraint(ans_model.model.routes, rule = rule3)

In [202]:
ans_model.model.C4 = pyo.Constraint(ans_model.model.routes, ans_model.model.transport, ans_model.model.startdays, ans_model.model.numbers, rule = rule4)

In [203]:
ans_model.model.C5 = pyo.Constraint(ans_model.model.transport, ans_model.model.startdays, ans_model.model.numbers, rule = rule5)

In [204]:
ans_model.model.C6 = pyo.Constraint(ans_model.model.routes, ans_model.model.transport, ans_model.model.startdays, ans_model.model.numbers, rule = rule6)

In [205]:
ans_model.model.C7 = pyo.Constraint(ans_model.model.routes, ans_model.model.transport, ans_model.model.startdays, ans_model.model.numbers, rule = rule7)

In [206]:
ans_model.model.C8 = pyo.Constraint(ans_model.model.routes, rule = rule8)

In [207]:
ans_model.model.C9 = pyo.Constraint(ans_model.model.warehouse_sites, ans_model.model.warehouse_sites, ans_model.model.transport, ans_model.model.startdays, ans_model.model.weights, ans_model.model.numbers, rule = rule9)

In [208]:
ans_model.model.C10 = pyo.Constraint(ans_model.model.warehouse_sites, ans_model.model.startdays, rule = rule10)

In [209]:
ans_model.model.C11 = pyo.Constraint(ans_model.model.customer_sites, ans_model.model.startdays, rule = rule11)

In [210]:
ans_model.model.C12 = pyo.Constraint(ans_model.model.warehouse_sites, ans_model.model.warehouse_sites, ans_model.model.transport, ans_model.model.startdays, ans_model.model.weights, ans_model.model.numbers, rule = rule12)

In [211]:
ans_model.model.C13 = pyo.Constraint(ans_model.model.customer_sites, ans_model.model.startdays, rule = rule13)

In [212]:
ans_model.model.C14 = pyo.Constraint(ans_model.model.warehouse_sites, ans_model.model.startdays, rule = rule14)

In [213]:
ans_model.model.C15 = pyo.Constraint(ans_model.model.routes, rule = rule15)

In [214]:
ans_model.model.C16 = pyo.Constraint(ans_model.model.routes, ans_model.model.transport, ans_model.model.startdays, ans_model.model.numbers, rule = rule16)

cost function

In [215]:
def satisfy_demand(model):
    cost = 0
    for i in range(len(model.routes)):
        for j in range(len(model.transport)):
            for k in range(len(model.startdays)):
                for l in range(len(model.numbers)):
                    cost = cost + model.x[i,j,k,l] * model.freight[i] * model.cost[j]
    for i in range(len(model.warehouse_sites)):
        for m in range(len(model.warehouse_sites)):
            for j in range(len(model.transport)):
                for k in range(len(model.startdays)):
                    for l in range(len(model.weights)):
                        for n in range(len(model.numbers)):
                            cost = cost + model.y[i,m,j,k,l,n] * l * model.cost[j]
                    
    return model.p * cost + (1-model.p) * model.final
    

objective: minimize the cost function

In [216]:
ans_model.model.obj=pyo.Objective(rule=satisfy_demand, sense=minimize)

solve using 'gurobi' solver

In [217]:
opt = SolverFactory('gurobi')

In [218]:
results = opt.solve(ans_model.model)

results

In [219]:
print(results)


Problem: 
- Name: x253090
  Lower bound: 977.5
  Upper bound: 977.5
  Number of objectives: 1
  Number of constraints: 385101
  Number of variables: 253090
  Number of binary variables: 0
  Number of integer variables: 253089
  Number of continuous variables: 1
  Number of nonzeros: 836093
  Sense: minimize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 6.314000129699707
  Error rc: 0
  Time: 11.787899017333984
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



results

In [227]:
cost = 0
for i in range(len(ans_model.model.routes)):
    for j in range(len(ans_model.model.transport)):
        for k in range(len(ans_model.model.startdays)):
            for l in range(len(ans_model.model.numbers)):
                cost = cost + pyo.value(ans_model.model.x[i,j,k,l]) * pyo.value(ans_model.model.freight[i]) * pyo.value(ans_model.model.cost[j])
                if (pyo.value(ans_model.model.x[i,j,k,l])):
                    print(f'{means_transport[j]} "{l+1}" goes from {routes_idx[i]} on day {k}')
for i in range(len(ans_model.model.warehouse_sites)):
    for m in range(len(ans_model.model.warehouse_sites)):
        for j in range(len(ans_model.model.transport)):
            for k in range(len(ans_model.model.startdays)):
                for l in range(len(ans_model.model.weights)):
                    for n in range(len(ans_model.model.numbers)):
                        cost = cost + pyo.value(ans_model.model.y[i,m,j,k,l,n]) * l * pyo.value(ans_model.model.cost[j])
                        if (pyo.value(ans_model.model.y[i,m,j,k,l,n])):
                            print(i,m,j,k,l,n)
                
# final = ans_model.model.p * cost + (1-ans_model.model.p) * pyo.value(ans_model.model.final)

print(f"minimum cost of transportation is {cost}")
print(f"corresponding minimum time taken is {pyo.value(ans_model.model.final)} days")
# print(f"minimum total value of transportation is {final}")

Ship "1" goes from A-D on day 0
Rail "3" goes from A-E on day 2
Ship "2" goes from B-E on day 0
Rail "1" goes from B-D on day 0
1 0 2 0 2 1
1 0 2 0 10 3
minimum cost of transportation is 1950.0
corresponding minimum time taken is 5.0 days
